In [1]:
import sys
import os
import pandas as pd
import glob
import numpy as np
from concat_files import concat_rrvs

# Valida CNPJ 
def validar_cnpj(s: pd.Series): 
    digitos = s.str.extract(r'(\d)(\d).(\d)(\d)(\d).(\d)(\d)(\d)/(\d)(\d)(\d)(\d)-').astype(np.uint8,errors="ignore")
    dv = s.str.extract(r'-(\d)(\d)').astype(np.uint8,errors="ignore")
    dig_1 = np.array([6,7,8,9,2,3,4,5,6,7,8,9 ],dtype=np.uint8)
    mult1 = np.sum(np.multiply(digitos,dig_1),axis=1).astype(np.uint16)
    res1 = np.remainder(mult1,11).astype(np.uint8)
    res1[res1==10] = 0
    digitos =  pd.concat([digitos,res1],axis=1)
    dig_2 = np.array([5,6,7,8,9,2,3,4,5,6,7,8,9],dtype=np.uint8)
    mult2 = np.sum(np.multiply(digitos,dig_2),axis=1).astype(np.uint16)
    res2 = np.remainder(mult2,11).astype(np.uint8)
    res2[res2==10] = 0
    digitos =  pd.concat([digitos,res2],axis=1)
    return ((dv[0] - res1) == 0) & ((dv[1] - res2) == 0)

# Formata CNPJ
def formatar_cnpj(s: pd.Series, check_val =False,on_error = "pass"): 
    s= s.apply(str).str.strip() # Transforma CPJS em string
    s.replace(to_replace="\W", value=r"", regex=True,inplace=True)
    s = s.str.pad(14,"left","0")  # Faz padding left com zeros
    s.replace(to_replace="(\d{2})(\d{3})(\d{3})(\d{4})(\d{2})", value=r"\1.\2.\3/\4-\5", regex=True,inplace=True)
    
    if check_val or on_error == "raise":
        invalidos  = ~validar_cnpj(s)
        quant_errors = invalidos.count()
        print(f"{quant_errors} CNPJs inválidos:")
        display(s[invalidos])
    if on_error == "raise" and quant_errors > 0:
        raise 
    
    return s

# Extrai as seguintes informações a partir do CEG: "Geração","Fonte","UF","ID","DV"
def extrair_info_ceg(s: pd.Series):
    ceg_info = s.str.extract("(\w{3}).(\w{2}).(\w{2}).(\d{6})-(\d{1})")
    ceg_info.columns = ["Geração","Fonte","UF","ID","DV"]
    ceg_info["ID"] = ceg_info["ID"].astype(np.uint16) 
    return ceg_info

In [2]:
# Caso atualize o arquivo, mudar caminho 
relatorio_outorgas_path = "./Inputs/relatorio_outorgas_19_08_22.xlsx"
relatorio_outorgas = pd.read_excel(relatorio_outorgas_path)

In [3]:
# Baixa relatório do outorgas. As colunas necessárias estão em "lista_coluna_outorgas"
lista_coluna_outorgas = ["Identificador da Usina","Nome da Usina","CEG"]
relatorio_outorgas=  pd.read_excel("./Inputs/relatorio_outorgas_19_08_22.xlsx")[lista_coluna_outorgas].rename(columns={"Nome da Usina":"Usina","Identificador da Usina":"ID"})

In [4]:
# Arquivos com os RRVs
path_files = "./Inputs/RRVs/"
file_paths = glob.glob(f"{path_files}/*.xlsx")

# Colunas dos RRVs
colunas = ["Evento","Mês/Ano","CEG do Empreendimento","Sigla Parcela - Usina", "(S) TOT_RV_D p,t,l,e,m", 
"Perfil Agente - Vendedor","Perfil Agente - Comprador","CNPJ Agente - Comprador","CNPJ Agente - Vendedor"]

In [5]:
rrv = concat_rrvs(file_paths,colunas)
rrv_copy = rrv.copy()

Lendo arquivo: ./Inputs/RRVs/Parquets/RRV_final_2021.gzip
Lendo arquivo: ./Inputs/RRVs/Parquets/RRV_final_2022_05.gzip
Lendo arquivo: ./Inputs/RRVs/Parquets/RRV_final_2022_06.gzip


In [6]:
rrv = rrv_copy.copy()


In [7]:
# Troca valores nulos por 0
rrv["(S) TOT_RV_D p,t,l,e,m"].fillna(0,inplace=True)

# Formata CNPJ
rrv["CNPJ Agente - Vendedor"] = formatar_cnpj(rrv["CNPJ Agente - Vendedor"])

# Formata data
rrv["Data"] = pd.to_datetime(rrv["Mês/Ano"],utc=False)

In [8]:
# Lista de colunas usadas para encontrar CEGs faltantes
lista_colunas_match = ["Sigla Parcela - Usina","Perfil Agente - Vendedor","CNPJ Agente - Vendedor"]
dict_parcela_ceg =   rrv[rrv["CEG do Empreendimento"].notna()].drop_duplicates(subset=lista_colunas_match)[lista_colunas_match + ["CEG do Empreendimento"]].drop_duplicates(subset=lista_colunas_match)
dict_parcela_ceg.loc[dict_parcela_ceg.isna().any(axis=1),:]

,Sigla Parcela - Usina,Perfil Agente - Vendedor,CNPJ Agente - Vendedor,CEG do Empreendimento


In [9]:
# Adiciona CEGs faltantes
cegs_identificados =  pd.merge(rrv[lista_colunas_match],dict_parcela_ceg,on=lista_colunas_match,how="left")["CEG do Empreendimento"]
cegs_identificados.index = rrv.index
rrv.loc[rrv["CEG do Empreendimento"].isna(),"CEG do Empreendimento"] = cegs_identificados


In [10]:
# Adiciona CEGs que foram identificados anteriormente
faltantes = pd.read_excel("./Intermediarios/Faltantes.xlsx")[lista_colunas_match + ["CEG do Empreendimento"]]
cegs_identificados =  pd.merge(rrv[lista_colunas_match],faltantes,on=lista_colunas_match,how="left")["CEG do Empreendimento"]
rrv.loc[rrv["CEG do Empreendimento"].isna(),"CEG do Empreendimento"] = cegs_identificados


In [11]:
# Gera tabela de falantes no mes
faltantes_mes = rrv[rrv["CEG do Empreendimento"].isna()].drop_duplicates(subset=lista_colunas_match)

faltantes_mes = faltantes_mes[lista_colunas_match+ ["CEG do Empreendimento"]]

display(faltantes_mes)

# Exporta para Excel e levanta erro para que o script seja interrompido e o usuário possa preencher a tabela
# Após preenchida, excecutar todas linhas subsequentes

if len(faltantes_mes.index) > 0:
    faltantes_mes.to_excel("./Intermediarios/Faltantes_mes.xlsx",index=False)
    raise ValueError("A tabela com CEGs faltantes foi exportada para identificação manual. Preencha o CEG com os casos que forem possível a identificação e continue a execução do script a partir da célula seguinte.")


,Sigla Parcela - Usina,Perfil Agente - Vendedor,CNPJ Agente - Vendedor,CEG do Empreendimento
187556,None,ARGON,21.642.355/0001-54,NaN
187557,None,BRASIL COM,13.145.928/0001-06,NaN
187558,None,ELETRONORTE,00.357.038/0001-16,NaN
187559,None,STATKRAFT,08.573.833/0001-53,NaN
187560,None,STIMA ENERGIA,25.099.255/0001-84,NaN
187561,None,TRADENER,02.691.745/0001-70,NaN
187562,None,ALUPAR,08.364.948/0001-38,NaN
187564,None,COPEL COM,19.125.927/0001-86,NaN
187565,None,SAFIRA COM,09.495.582/0001-07,NaN
187587,None,ELETRON,15.087.610/0001-41,NaN


ValueError: A tabela com CEGs faltantes foi exportada para identificação manual. Preencha o CEG com os casos que forem possível a identificação e continue a execução do script a partir da célula seguinte.

In [12]:
# Carrega arquivo com faltantes_mes e exclui linhas que não foram identificadas
faltantes_mes = pd.read_excel("./Intermediarios/Faltantes_mes.xlsx").dropna(subset="CEG do Empreendimento")[lista_colunas_match + ["CEG do Empreendimento"]]
# Adiciona ao arquivo faltantes os casos que foram identificados e constam no arquivo faltantes_mes
faltantes = pd.concat([faltantes,faltantes_mes],ignore_index=True).drop_duplicates(subset=lista_colunas_match)[lista_colunas_match + ["CEG do Empreendimento"]]
faltantes.to_excel("./Intermediarios/Faltantes.xlsx",index=False)

In [13]:
# Adiciona CEGs identificados
cegs_identificados =  pd.merge(rrv[lista_colunas_match],faltantes_mes,on=lista_colunas_match,how="left",validate="many_to_one")["CEG do Empreendimento"]
cegs_identificados.index = rrv.index
rrv.loc[rrv["CEG do Empreendimento"].isna(),"CEG do Empreendimento"] = cegs_identificados

In [14]:
# Rendas com CEG não indentificado
rrv_nao_ident = rrv.loc[rrv["CEG do Empreendimento"].isna()][lista_colunas_match + ["(S) TOT_RV_D p,t,l,e,m","Data"]]
rrv_nao_ident = rrv_nao_ident[rrv_nao_ident["(S) TOT_RV_D p,t,l,e,m"] > 0]

In [15]:
# Exclui linhas sem CEG
rrv.dropna(subset="CEG do Empreendimento",inplace=True)
rrv["ID"] = extrair_info_ceg(rrv["CEG do Empreendimento"])["ID"]
display(rrv.isna().any())

Evento                       False
Mês/Ano                      False
CEG do Empreendimento        False
Sigla Parcela - Usina        False
(S) TOT_RV_D p,t,l,e,m       False
Perfil Agente - Vendedor     False
Perfil Agente - Comprador    False
CNPJ Agente - Comprador      False
CNPJ Agente - Vendedor       False
Data                         False
ID                           False
dtype: bool

In [16]:
# Colunas para exportação
colunas_export = ["Data","ID","Sigla Parcela - Usina","CNPJ Agente - Vendedor","(S) TOT_RV_D p,t,l,e,m"]
rrv = rrv[colunas_export]

In [17]:
# Baixa arquivos de 2020 que foram gerados no script Identificacao_usinas_2020
rrv_2020 = pd.read_parquet("./Intermediarios/RV_17_2020.gzip")[colunas_export]
rrv_nao_ident_2020 = pd.read_parquet("./Intermediarios/rrv_nao_ident_2020.gzip")[colunas_export]
rrv_preliminar = pd.read_parquet("./Intermediarios/RV_17_2020.gzip")[colunas_export]

# Agrega dados atuais com os de 2020
if rrv[rrv.Data.isin(rrv_2020.Data)].size > 0:
    raise ValueError("Dados de 2020 já estão em ./Intermediarios/RV_17_2020.gzip")
else:
    rrv = pd.concat([rrv_2020,rrv],axis=0,ignore_index=True)
    rrv_nao_ident = pd.concat([rrv_nao_ident,rrv_nao_ident_2020],axis=0,ignore_index=True).sort_values(by="Data")
del rrv_nao_ident["ID"]

In [18]:
# Gera tabela dimensão para o relatório em PowerBI
rrv_dimensao = rrv.drop_duplicates(subset="ID")[["ID"]].reset_index(drop=True)
rrv_dimensao = pd.merge(rrv_dimensao,relatorio_outorgas,on="ID",validate="one_to_one")
datas = rrv.drop_duplicates(subset="Data")[["Data"]].reset_index(drop=True)
display(rrv_dimensao.isna().any())
display(datas.isna().any())


ID       False
Usina    False
CEG      False
dtype: bool

Data    False
dtype: bool

In [19]:
# Salva dataframes em arquivo excel
with pd.ExcelWriter('./Outputs/P_BI_base_RRV.xlsx',datetime_format="DD/MM/YYYY",date_format="DD/MM/YYYY") as writer:
    datas.to_excel(writer,sheet_name='Datas',index=False)
    rrv_dimensao.to_excel(writer,sheet_name='Tabela Dimensão',index=False)
    rrv.to_excel(writer,sheet_name='RRV Final',index=False)
    rrv_preliminar.to_excel(writer,sheet_name='RRV Preliminar',index=False)
    rrv_nao_ident.to_excel(writer,sheet_name='RRV no_ident',index=False)